<a href="https://colab.research.google.com/github/MaguireMaName/Coursera_Capstone/blob/master/Machine_Learning_w_Python_Segment_%26_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install geocoder
#!pip install requests

In [0]:
# bring in dependencies 
import geocoder
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests as rq
import requests

## Machine Learning with Python: Dataframe of postal code, neighborhood, & borough
*For the Applied Data Science Capstone Project*

In [6]:
# define url for scraping and print

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
print(url)

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


In [0]:
response = rq.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

#print(soup)

In [0]:
table = soup.find('table', {'class':'wikitable sortable'}).tbody
#print(table)

rows = table.find_all('tr')

columns = [v.text.replace('\n','') for v in rows[0].find_all('th')]

df_a = pd.DataFrame(columns=columns)

for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    if len(tds) ==4:
      values = [tds[0].text, tds[1].text,'', tds[2].text, tds[3].text.replace('\n','').replace('\xa0','')]
    else:
      values = [td.text.replace('\n','').replace('\xa0','') for td in tds]
    
    df_a = df_a.append(pd.Series(values, index=columns), ignore_index=True)


In [9]:
# dimensions before aggregation

df_a.shape

(288, 3)

In [0]:
# aggregate data

df_b = df_a.groupby(['Postcode','Borough']).agg(lambda x: x.tolist()).reset_index()

In [12]:
df_b.shape

(180, 3)

In [13]:
# check results

df_b.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,[Not assigned]
1,M1B,Scarborough,"[Rouge, Malvern]"
2,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]"
3,M1E,Scarborough,"[Guildwood, Morningside, West Hill]"
4,M1G,Scarborough,[Woburn]


In [0]:
# where neighbourhood is not assigned, replace it with borough

df_b['Neighbourhood'] = np.where(df_b['Neighbourhood'] == "Not assigned", df_b['Borough'], df_b['Neighbourhood'])


In [15]:
# exception table

x_neighbourhood = df_b.loc[(df_b['Neighbourhood'] == "Not assigned")]
x_neighbourhood.shape

(0, 3)

In [16]:
# exception table

x_borough = df_b.loc[(df_b['Borough'] == "Not assigned")]
x_borough.shape

(77, 3)

In [0]:
# 77 boroughs not assigned. Don't process obs. where borough = 'Not assigned'

df_c = df_b.drop(df_b[df_b.Borough == "Not assigned"].index)


In [19]:
# exception table

x_borough = df_c.loc[(df_b['Borough'] == "Not assigned")]
x_borough.shape

(0, 3)

In [20]:
# dimensions after aggregation

df_c.shape

(103, 3)

In [0]:
# load in lat and lon info

df_geo = pd.read_csv('Geospatial_Coordinates.csv')

In [23]:
# check geo load

df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [0]:
df_geo.rename(columns = {"Postal Code": "Postcode"}, 
                                 inplace = True) 

In [0]:
# join geo with neighbourhood data

df_d = pd.merge(df_geo, df_c, on='Postcode')

In [26]:
# check dimensions

df_d.shape

(103, 5)

In [34]:
# check data

df_d.head()

,Postcode,Latitude,Longitude,Borough,Neighbourhood
98,M9N,43.706876,-79.518188,York,[Weston]
99,M9P,43.696319,-79.532242,Etobicoke,[Westmount]
100,M9R,43.688905,-79.554724,Etobicoke,"[Kingsview Village, Martin Grove Gardens, Rich..."
101,M9V,43.739416,-79.588437,Etobicoke,"[Albion Gardens, Beaumond Heights, Humbergate,..."
102,M9W,43.706748,-79.594054,Etobicoke,[Northwest]


In [28]:
CLIENT_ID = 'KL5SVGOS40RKZBQK4G1VXYBKBICWCDQL2NMCASHFYER432SS'
' # your Foursquare ID'
CLIENT_SECRET = '1A5KPYJQIATH0SDZXPPZ5YK0SHLBYVEGPER5AAIIMDXLZ0AB' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KL5SVGOS40RKZBQK4G1VXYBKBICWCDQL2NMCASHFYER432SS
CLIENT_SECRET:1A5KPYJQIATH0SDZXPPZ5YK0SHLBYVEGPER5AAIIMDXLZ0AB


In [52]:
# will explore features of neighbourhood in Toronto only

df_d[df_d['Borough'].str.contains("Toronto")]

,Postcode,Latitude,Longitude,Borough,Neighbourhood
37,M4E,43.676357,-79.293031,East Toronto,[The Beaches]
41,M4K,43.679557,-79.352188,East Toronto,"[The Danforth West, Riverdale]"
42,M4L,43.668999,-79.315572,East Toronto,"[The Beaches West, India Bazaar]"
43,M4M,43.659526,-79.340923,East Toronto,[Studio District]
44,M4N,43.728020,-79.388790,Central Toronto,[Lawrence Park]
45,M4P,43.712751,-79.390197,Central Toronto,[Davisville North]
46,M4R,43.715383,-79.405678,Central Toronto,[North Toronto West]
47,M4S,43.704324,-79.388790,Central Toronto,[Davisville]
48,M4T,43.689574,-79.383160,Central Toronto,"[Moore Park, Summerhill East]"
49,M4V,43.686412,-79.400049,Central Toronto,"[Deer Park, Forest Hill SE, Rathnelly, South H..."
